In [ ]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [ ]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [ ]:
formula = """
    ClaimCount ~ 
    ms(VehAge, df=4, increasing=True)
"""

result = rs.glm(
    formula,
    train_data, 
    family="poisson", 
    offset="Exposure").fit(
)

print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406434
Method:              IRLS            Df Model:                     4
Scale:               1.0000          Iterations:                  25

Log-Likelihood:         -121185.5141 Deviance:                200823.3785
AIC:                     242381.0282 Null Deviance:           132328.3090
BIC:                     242435.6042 Pearson chi2:              396324.82
Converged:           False          

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -1.2062     0.0182  -66.249  <0.0001   [ -1.2419,  -1.1706]  ***
ms(VehAge, 1/4, +)     0.0000     0.0203    0.000  

In [ ]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': '\n    ClaimCount ~ \n    TE(Region) + \n    BonusMalus + \n    I(BonusMalus ** 2) + \n    ms(VehAge, df=4) + \n    pos(DrivAge) + \n    pos(I(DrivAge ** 2)) + \n    TE(VehBrand) + \n    TE(Area), \n', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 12, 'df_resid': 406427, 'converged': False, 'iterations': 25}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=83838.34, loss=0.502191, deviance=204109.94, log_likelihood=-122828.79, aic=245681.58, gini=0.1587, auc=0.5793, ae_ratio=0.2553, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1717.56, actual=0.609003, predicted=0.261613, ae_ratio=2.3279), DecileMetrics(decile=2, n=40643, exposure=3751.58, actual=0.302806, predicted=0.297825, ae_ratio=1.0167), DecileMetrics(decile=3, n=40643, exposure=7606.59, actual=0.181027, predicted=0.285224, ae_ratio=0.6347), DecileMetrics(de

In [ ]:
result.predict(train_data)

array([0.04571342, 0.47704766, 0.01262805, ..., 0.01359945, 0.21464508,
       0.18741592], shape=(406439,))

In [ ]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"},
        "VehAge": {"type": "bs", "df": 4, "monotonicity": "increasing"},
        "DrivAge": {"type": "linear", "monotonicity": "increasing"},
        "DrivAge2": {"type": "expression", "expr": "DrivAge ** 2", "monotonicity": "increasing"},
        "VehBrand": {"type": "target_encoding"},
        "Area": {"type": "target_encoding"},
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()